# Создание Телеграм-ботов. Боты с базами данных.

Сегодня мы с вами выясним, что такое базы данных, и зачем они нужны в Телеграм-ботах. Кроме этого, научимся работать с медиафайлами в телеграме по `file_id`.

---

Ссылки:
* Описание всех методов и классов Telegram Bot API: https://core.telegram.org/bots/api
* Гайд по созданию Телеграм-ботов: https://mastergroosha.github.io/telegram-tutorial/
* О `shelve`: https://metanit.com/python/tutorial/4.6.php
* О `SQLite`: https://python-scripts.com/sqlite
* Скачать `DB Browser for SQLite`: https://sqlitebrowser.org/dl/

Боты, которых мы писали ранее, хранили все данные о пользователях в оперативной памяти. Этот метод плох, потому что, если перезапустить бота, то вся информация будет утеряна.  

Из-за этого, важную информацию нужно хранить в файлах на диске. Как это сделать мы сейчас и обсудим.  

*Сделаю важное замечание: работа с файлами на диске в тысячи раз медленнее, чем с файлами в оперативной памяти (RAM). Поэтому не увлекайтесь - храните на диске только информацию, потеря которой критична.*

# Shelve

Самый очевидный вариант - создать какой-нибудь `txt`-файлик, и в него все напрямую записать через метод `open`. Так можно сделать, но тогда работа с данными будет очень неудобной: нам придется написать много кода, чтобы доставать из файла информацию, которая нужна в данный момент.  

Подход, когда информация просто записывается в файл, удобно реализована в модуле `shelve`. Интерфейс работы с данными в этом модуле аналогичен работе со словарями. Единственное ограничение: ключ должен быть строкой. А значение - не может изменяться частично (т.е. нельзя добавить **один** элемент в список, только записать весь список заново).

Пример использования `shelve`-хранилища на чтение и запись.

In [1]:
import shelve
 
FILENAME = "capitals"
with shelve.open(FILENAME, 'c') as capitals:    # Перед использованием - shelve-хранилище надо открыть
    capitals["Great Britain"] = "London"
    capitals["France"] = "Paris"
    capitals["Germany"] = "Berlin"
    capitals["Spain"] = "Madrid"
 
with shelve.open(FILENAME) as capitals:
    print(capitals["Great Britain"])
    print(capitals["Spain"])
    
    del capitals["Spain"]
    print("Spain" in capitals)

London
Madrid
False


Есть 4 режима открытия shelve-хранилища:  

* **c:** файл открывается для чтения и записи (значение по умолчанию). Если файл не существует, то он создается.
* **r:** файл открывается только для чтения.
* **w:** файл открывается для записи (если файла не было - кинет ошибку).
* **n:** файл открывается для записи Если файл не существует, то он создается. Если он существует, то он перезаписывается

# SQLite

Для более важной и сложнее структурированной информации используют базы данных (database).  

* Базы данных по-особому хранят данные. Из-за этого доступ к информации при больших размерах базы сравнительно быстр.
* В базах данных информация серьезнее защищается от непредвиденных ситуаций: например, от исключений во время чтения или записи.  
* С базой данных одновременно могут работать несколько программ на чтение и запись (с `shelve` так можно делать только на чтение).  
* Базы данных поддерживают язык запросов `SQL`, который позволяет находить и изменять информацию по определенным правилам.

И это далеко не все.

Я расскажу про базу данных `SQLite`. Ее можно использовать с программами на Питоне без скачивания дополнительных модулей.


## Графический интерфейс

С базой данных можно работать программно. Но задать ее первоначальный вид и визуализировать ее содержимое можно через утилиту `DB Browser for SQLite`.  
Скачать ее можно здесь: https://sqlitebrowser.org/dl/

<table><tr><td><img src="БД-1.jpg" style="height:350px;"/></td> <td><img src="БД-2.jpg" style="height:350px;"/></td></tr> <tr> <td><img src="БД-3.jpg" style="height:250px;"/></td> <td><img src="БД-4.jpg" style="height:250px;"/></td> </tr></table>

Алгоритм создания базы данных прост:  
1. Жмем на `Новая база данных` в левом верхнем углу.
2. Выбираем, где сохранить базу данных.  
В следующем окне прописываем название таблицы (его дальше будем использовать в коде) и столбцы таблицы (для них указываем тип данных, который в них будет храниться).
3. Чтобы добавить в даблицу значение - жмем `Добавить запись`.
4. Кликаем по ячейкам новой пустой строки и заполняем их.
5. Жмем `Записать изменения` (рядом с `Новая база данных`).

## Работа с SQLite через Python
### Создание таблицы


In [2]:
import sqlite3  # Подключаем модуль
 
connection = sqlite3.connect("mydatabase.db")  # или в скобочках написать :memory: чтобы сохранить в RAM.
cursor = connection.cursor()  # Объект для работы с БД.

# Создание таблицы
cursor.execute("""CREATE TABLE music
                  (id integer, Исполнитель text,
                   Композиция text)
               """)

# SQLite поддерживает пять типов данных: null, integer, real, text и blob.

Здесь мы создали таблицу в базе данных с теми же столбцами, что и через графический интерфейс. Если файла `mydatabase.db` на момент вызова функции не было - он создастся автоматически.  

Если мы попробуем вызвать эту функцию еще раз, то возникнет ошибка, потому что в базе данных `mydatabase.db` уже есть таблица `music`.

Основная функция для работы с `SQLite` - это `cursor.execute(command)`. Здесь `command` - это строка с SQL-запросом.

### Заполнение таблицы

In [7]:
# Вставляем данные в таблицу
cursor.execute("INSERT INTO music VALUES (1, 'Panic! At The Disco', 'Vegas Lights')")

# Вставляем множество данных в таблицу используя безопасный метод "?"
music = [(2, 'Arctic Monkeys', 'I Bet You Look Good On The Dancefloor'),
         (3, 'Kanye West', 'Wolves'),
         (4, 'Katy Perry', 'Roar')]     # В списке лежат строки, которые будут добавлены. 
                                        # Каждый элемент - кортеж из элементов строки.
 
cursor.executemany("INSERT INTO music VALUES (?,?,?)", music)  # Число вопросиков - число столбцов в таблице.
                                                               # На место вопросиков встанут элементы кортежа.

# Считаем количество строк в таблице
amount_of_rows = cursor.execute("SELECT COUNT (*) FROM music").fetchone()[0]
print(amount_of_rows)

# Сохраняем изменения
connection.commit()

8


### Редактирование таблицы

In [4]:
# Заполняет поле "Композиция" значением "Jesus Walks" в строчке, где "Исполнитель" - "Kanye West"
sql = "UPDATE music SET Композиция = 'Jesus Walks' WHERE Исполнитель = 'Kanye West'"
cursor.execute(sql)

# Удаляет строчку, где исполнитель - "Katy Perry"
sql = "DELETE FROM music WHERE Исполнитель = 'Katy Perry'"
cursor.execute(sql)

# Удаляет все строчки из таблицы
sql = "DELETE FROM music"
cursor.execute(sql)

connection.commit()

### Запросы к таблице

In [14]:
# Если раскомментировать строку ниже, то строки вернутся как объекты Row с интерфейсом словаря.
# connection.row_factory = sqlite3.Row  
 
# Выбирает из таблицы строки, где исполнитель - то, что передаем вторым параметром в функцию execute()
# * означает, что будут выбраны все элементы из строки
sql = "SELECT * FROM music WHERE Исполнитель=?"
cursor.execute(sql, ("Kanye West",))
print(cursor.fetchone())  # Возвращает все строки. Если использовать fetchone(), то вернет первую встретившуюся.


# Сначала пишет номер строки, потом все элементы строки. Все результаты сортируются по исполнителю.
for row in cursor.execute("SELECT rowid, * FROM music ORDER BY Исполнитель"):
    print(row)
 

# Выбирает строчки, где композиция содержит 'The'.
sql = "SELECT * FROM music WHERE Композиция LIKE '%THE%'"
cursor.execute(sql)
print(cursor.fetchall())

(3, 'Kanye West', 'Wolves')
(2, 2, 'Arctic Monkeys', 'I Bet You Look Good On The Dancefloor')
(6, 2, 'Arctic Monkeys', 'I Bet You Look Good On The Dancefloor')
(3, 3, 'Kanye West', 'Wolves')
(7, 3, 'Kanye West', 'Wolves')
(4, 4, 'Katy Perry', 'Roar')
(8, 4, 'Katy Perry', 'Roar')
(1, 1, 'Panic! At The Disco', 'Vegas Lights')
(5, 1, 'Panic! At The Disco', 'Vegas Lights')
[(2, 'Arctic Monkeys', 'I Bet You Look Good On The Dancefloor'), (2, 'Arctic Monkeys', 'I Bet You Look Good On The Dancefloor')]


# Работа с медиа через `file_id`

Чтобы экономить трафик, память и время - в Телеграм для отправки медиа можно отправлять не сами файлы, а их уникальный `file_id`. При условии, что этот файл был когда-то отправлен на сервера Телеграма, конечно.  

**Важно!** Для разных ботов `file_id` одних и тех же медиафайлов различаются. Сам `file_id` - питоновская строка.


## Получение `file_id`

Чтобы получить `file_id` медиафайла нужно посмотреть на переменную `message`, соответствующую сообщению, в котором был отправлен исходный файл.  

В зависимости от типа контента в сообщении будут присутствовать поля:
* photo
* audio
* video
* voice  

и другие. Про остальные поля можно почитать здесь: https://core.telegram.org/bots/api#message  

В каждом из этих полей, кроме `photo`, есть поле `file_id`. Оно нам и нужно.  
Т.е., чтобы достать `file_id` аудиосообщения - нам понадобится написать `message.audio.file_id`.  

С полем `photo` ситуация немного интереснее. Поле `photo` - это список объектов `PhotoSize`. Каждый из этих объектов соответствует этому же изображению, но в разных разрешениях. Это сделано, чтобы на маленьких экранах не загружались чересчур большие фотографии. В списке качество фотографий идет по возрастанию. Количество элементов в списке зависит от разрешения исходного изображения (чем оно больше, тем больше градаций, тем больше элементов в списке).

`file_id` нужно смотреть непосредственно у объекта `PhotoSize`. При *отправке* изображения по `file_id` нет разницы, у какого из `PhotoSize` мы его возьмем. Но при *скачивании* изображения по `file_id` - скачается изображение с соответствующим разрешением.  

Т.е., чтобы достать `file_id` фотографии с наихудшим разрешением - нужно написать `message.photo[0].file_id`.  

In [ ]:
@bot.message_handler(content_types=['text'])
def get_file_id(message):
    bot.send_message(message.chat.id, "привет")
    with open("Koala.jpg", "rb") as photo:
        msg = bot.send_photo(message.chat.id, photo)
        print(msg)
        bot.send_message(message.chat.id, msg.photo[2].file_id)

## Скачивание файла по `file_id`

Для скачивания файла понадобится функция `bot.get_file(file_id)`. Она по `file_id` вернет объект `File`.  

Ваш файл можно скачать по ссылке: `https://api.telegram.org/file/bot<token>/<file_path>`. Здесь вместо `<token>` нужно указать токен вашего бота, а вместо `<file_path>` - поле `bot.get_file(file_id).file_path`.  

В скачивании объекта по ссылке вам поможет функция `requests.get(link).content` из модуля `requests`.  

В сборе код скачивания файла и записи его в файл выглядит так:

In [ ]:
def download_file(file_id, file_name):
    link = 'https://api.telegram.org/file/bot{}/{}'.format(config.TOKEN, bot.get_file(file_id).file_path)
    to_write = requests.get(link).content
    with open(file_name, "wb") as file:
        file.write(to_write)

# Пишем бота-викторину "Угадай мелодию"

In [ ]:
# Код в PyCharm проекте.